In [42]:
import pandas as pd
import requests
from pprint import pprint
from config import g_key

In [30]:
ziplatlng = pd.read_csv('us-zip-code-latitude-and-longitude.csv', delimiter = ';')

In [37]:
zipGA = ziplatlng.loc[ziplatlng['State']=='GA']
zipGA

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
15,31150,Atlanta,GA,33.844371,-84.474050,-5,1,"33.844371,-84.47405"
16,31646,Saint George,GA,30.518903,-82.080630,-5,1,"30.518903,-82.08063"
17,31826,Shiloh,GA,32.803518,-84.693840,-5,1,"32.803518,-84.69384"
65,30334,Atlanta,GA,33.702657,-84.439127,-5,1,"33.702657,-84.439127"
70,31606,Valdosta,GA,30.802411,-83.196240,-5,1,"30.802411,-83.19624"
...,...,...,...,...,...,...,...,...
42987,31402,Savannah,GA,31.971394,-81.071560,-5,1,"31.971394,-81.07156"
43085,30152,Kennesaw,GA,34.002147,-84.633920,-5,1,"34.002147,-84.63392"
43117,30655,Monroe,GA,33.786041,-83.699010,-5,1,"33.786041,-83.69901"
43172,30376,Atlanta,GA,33.844371,-84.474050,-5,1,"33.844371,-84.47405"


In [38]:
city_name = 'Atlanta'
address = 31150
country_name = 'US'
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'locality={0}&address={0}&country={1}&key={2}').format(city_name,country_name, g_key)

In [41]:
geo_data = requests.get(target_url).json()

In [101]:
def countySearch(json):
    for locales in json['results'][0]['address_components']:

        if locales['types'][0] == 'administrative_area_level_2':
            county = locales['long_name']
    return county
    


In [105]:
counties = []

for row in zipGA.iterrows():
    city_name = row[1]['City']
    address = row[1]['Zip']
    country_name = 'US'
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'locality={0}&address={0}&country={1}&key={2}').format(city_name,country_name, g_key)
    geodata = requests.get(target_url).json()
    
    try:
        county = countySearch(geodata)
    except:
        county = 'n/a'
    
    counties.append(county)



In [106]:
zipGA['County'] = counties

C:\Users\msflo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [108]:
zipGA.to_csv('ZipCounty.csv')